## Statistics extraction from Imaris

#### Expected data layout
* Condition
  * description_Statistics
    * filename_feature.csv

#### Load required libraries

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tools.tools as tls
import imreader.imreader as imr
import tools.globalsettings as gbs
import csv
logr = gbs.lgr

#### Configure paths

In [2]:
#inpath = '/home/bcardoen/mountcedar/project/data/nabi-robert-ivan/RL.ZikaEr.001/Imaris_Export_20181126/Imaris_ER_Analysis'
# outpath = '/home/bcardoen/SFUVault/Repositories/nbr/output'
#inpath = '/home/bcardoen/Desktop/Rory/Imaris_ER_Analysis'
inpath = '/home/bcardoen/Desktop/Rory/ER_with_NS2B'
outpath = '/home/bcardoen/SFUVault/Repositories/imaris_reader/output'
assert(os.path.exists(inpath))
assert(os.path.exists(outpath))

#### Build a virtual directory tree

In [3]:
tree = tls.treedir(inpath)
conditions = [k for (k,v) in tree.items() if isinstance(v, dict)]
print('\t Have conditions: '.format(conditions))
for c in conditions:
    print('\t Condition {}'.format(c))

	 Have conditions: 
	 Condition CER_with_dsRNA
	 Condition PER_with_NS2B
	 Condition PER_with_dsRNA
	 Condition PER_without_dsRNA
	 Condition ER_with_dsRNA
	 Condition PER
	 Condition CER_with_NS2B
	 Condition CER
	 Condition CER_without_dsRNA


#### Decode the directory structure into its separate parts (cellnr, date, ...)

In [4]:
data = {}
for C in conditions:
    print('\t Parsing condition {}'.format(C))
    for dname, ct in tree[C].items():
        if isinstance(ct, dict):
            if dname.endswith('Statistics'):
                result = imr.parse_dirname(dname)
                result['condition'] = C
                key = tuple((k, result[k]) for k in sorted(result.keys()))
                if key not in data:
                    data[key] = (dname,ct)
                else:
                    logr.error('ERROR : {} is duplicate'.format(key))
                    logr.error('ERROR : Old entry by fname {}'.format(data[key]))
                    logr.error('ERROR : new entry by fname {}'.format(dname))
                    logr.error('ERROR : Treatment = {}'.format(C))
                    raise ValueError
                    

	 Parsing condition CER_with_dsRNA
	 Parsing condition PER_with_NS2B
	 Parsing condition PER_with_dsRNA
	 Parsing condition PER_without_dsRNA
	 Parsing condition ER_with_dsRNA
	 Parsing condition PER
	 Parsing condition CER_with_NS2B
	 Parsing condition CER
	 Parsing condition CER_without_dsRNA


#### Load all the features for the decoded structure (e.g. read the csv files)

In [5]:
aggregated = {}
for key, (dname,ct) in data.items():
    condition = key[1][1]
    fp = os.path.join(inpath, condition, dname)
    if os.path.exists(fp):
        contents = imr.parse_directory(fp)
        if key in aggregated:
            logr.error('Duplicate key')
            raise ValueError
        aggregated[key] = contents
    else:
        logr.error('No such path {}'.format(fp))
        break
logr.info('\t A total of {} files parsed'.format(len(aggregated)))

[<ipython-input-5-bb1af808c353>:14 -             <module>() ] 	 A total of 117 files parsed


#### Write the output

In [6]:
imr.outwriter(aggregated, outpath, 'table.csv')

[imreader.py:151 -            outwriter() ] 	 Wrote 34 columns and 117 rows
